In [1]:
import sys
import pandas as pd
import json
import os
import pickle
import re
import difflib
from difflib import SequenceMatcher
import jellyfish
pd.set_option('display.max_columns', 100)

In [2]:
import spotipy
import spotipy.util as util

In [4]:
user_name = "Pillsbury Darboy"
client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
client_secret = "53634fc86e8a4f71b7459197084e3cdf"
redirect_uri = "http://localhost/"
scope = 'user-library-read'
from spotipy.util import prompt_for_user_token
token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
from spotipy import Spotify
spotify = Spotify(auth=token)

In [5]:
class SetlistGenerator_Spotify(object):

    def __init__(self, seed_tracks='', seed_genres='', seed_artists=''):
        self.seed_genres = seed_genres
        self.seed_tracks = seed_tracks
        self.seed_artists = seed_artists
        
        self.AuthorizeSpotify()
        
        #Setlist starts with seed_track
        self.set_list = self.get_track_metadata_single()
        
        self.genres = ['chicago-house', 'club', 'dancehall', 'deep-house', 'detroit-techno', 'dub', 'edm',
         'electro', 'electronic', 'groove', 'house', 'idm', 'industrial', 'minimal-techno', 'progressive-house', 
          'soul', 'techno', 'trance']
        self.moods = ['happy', 'sad']
        
        self.track_pool = self.getRecommendations()
        
    def getRecommendations(self):
        # Query spotify for recommendations based on a track
        recommendations = self.spotify.recommendations(seed_artists=self.seed_artists, 
                                                       seed_genres=self.seed_genres, 
                                                       seed_tracks=self.seed_tracks, 
                                                       limit = 50)
        # Convert spotify data into readable format
        metadata = self.get_track_metadata(recommendations)
        # Retrieve audio features for spotify data
        audio_features = pd.DataFrame(self.spotify.audio_features(self.get_track_ids(recommendations)))
        df = metadata.merge(audio_features)
        
        #Correct key + mode data 
        df = self.key_mode_convert(df)
        #del df['type']
        #del df['analysis_url']
        #del df['disc_number']
        #del df['track_number']
        return(df)

    def getNextTrack(self, method = 0, min_dance=0.6):
        candidates = self.track_pool
        current_song_key = int(self.set_list[-1:]['camelot'])
        if method == 0:
            candidates = self.filter_by_camelot(df = candidates, 
                                                song_key_camelot = current_song_key)
            candidates = self.filter_by_popularity(candidates)
            candidates = self.filter_by_danceability(candidates, min_dance=min_dance)
        else:
            pass
        self.set_list = self.set_list.append(candidates.sample(1))
        self.set_list = self.set_list.reset_index(drop=True)
        
        return(self.set_list)
    
    ############################################
    ######      SEED     UTILITIES        ######
    ############################################
    
    def changeSeedTracks(self, use_last = 2):
        self.seed_tracks = list(self.set_list['id'][-use_last:])
        
    def changeSeedGenres(self, new_genres):
        self.seed_genres = new_genres
        
    def changeSeedArtists(self, new_artists):
        self.seed_genres = artists
    
    
    ############################################
    ######     TRACK   POOL   UTILITIES   ######
    ############################################
        
    def getTrackPool(self):
        return self.track_pool
    
    def expandTrackPool(self):
        self.track_pool = self.track_pool.append(self.getRecommendations())
        self.cleanTrackPool()
        
        print('Expanded track pool')
    
    def cleanTrackPool(self):
        self.track_pool = self.track_pool[-self.track_pool['id'].isin(list(self.set_list['id']))]
        self.track_pool = self.track_pool.drop_duplicates()
        self.track_pool = self.track_pool.reset_index(drop=True)

        
    ############################################
    ######    TRACK SELECTION UTILITIES   ######
    ############################################
        
    def closest_keys(self, camelot):
        closest = []
        closest.append(camelot)
        closest.append((camelot + 1)%12)
        closest.append((camelot - 1)%12)
        return(closest)
    
    def filter_by_camelot(self, df, song_key_camelot):
        filtered = []
        for i in self.closest_keys(song_key_camelot):
            filtered.append(pd.DataFrame(df[df['camelot'] == i]))
        return(pd.concat(filtered))
        
    def filter_by_popularity(self, df, min_pop=0, max_pop=100):
        df = df[df['popularity'] >= min_pop]
        df = df[df['popularity'] <= max_pop]
        return(df)
    
    def filter_by_tempo(self, df, min_tempo = 90, max_tempo = 145):
        df = df[df['tempo'] >= min_tempo]
        df = df[df['tempo'] <= max_tempo]
        return(df)
    
    def filter_by_valence(self, df=None, min_valence = 0, max_valence = 1):
        df = self.track_pool
        df = df[df['valence'] >= min_valence]
        df = df[df['valence'] <= max_valence]
        return(df)
    
    def filter_by_danceability(self, df=None, min_dance = 0, max_dance = 1):
        df = self.track_pool
        df = df[df['danceability'] >= min_dance]
        df = df[df['danceability'] <= max_dance]
        return(df)
    
    def filter_by_energy(self, df=None, min_energy = 0, max_energy = 1):
        df = self.track_pool
        df = df[df['energy'] >= min_energy]
        df = df[df['energy'] <= max_energy]
        return(df)
    
    def filter_by_liveness(self, df=None, min_liveness = 0, max_liveness = 1):
        df = self.track_pool
        df = df[df['liveness'] >= min_liveness]
        df = df[df['liveness'] <= max_liveness]
        return(df)
    
    def filter_by_acousticness(self, df=None, min_acoustic = 0, max_acoustic = 1):
        df = self.track_pool
        df = df[df['acousticness'] >= min_acoustic]
        df = df[df['acousticness'] <= max_acoustic]
        return(df)
    
    def filter_by_instrumentalness(self, df=None,
                                   min_instrumentalness = 0, 
                                   max_instrumentalness = 1):
        df = self.track_pool
        df = df[df['instrumentalness'] >= min_instrumentalness]
        df = df[df['instrumentalness'] <= max_instrumentalness]
        return(df)



    ############################################
    ######        METADATA UTILITIES      ######
    ############################################
    
    def get_track_metadata(self, recommendations):
        tracklist_frames = []
        for track in recommendations['tracks']:
            tracklist_frames.append({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']})
        return(pd.DataFrame(tracklist_frames))

    def get_track_metadata_single(self):
        track_id = self.seed_tracks[0]
        track = self.spotify.track(track_id)
        metadata = pd.DataFrame({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']}, index =[0])
        audio_features = pd.DataFrame(self.spotify.audio_features(track_id))
        df = metadata.merge(audio_features)
        df = self.key_mode_convert(df)
        return(df)
    
    def get_track_ids(self, recommendations):
        id_list = []
        for spotify_id in recommendations['tracks']:
            id_list.append(spotify_id['id'])
        return(id_list)
    
    def key_mode_convert(self, recommendations):
        df = recommendations
        camelot_list = []
        for i in range(0, len(df)):
            if df['mode'][i] == 1:
                if (df['key'][i] == 0):
                    camelot_list.append(7)
                if (df['key'][i] == 1):
                    camelot_list.append(2)
                if (df['key'][i] == 2):
                    camelot_list.append(9)
                if (df['key'][i] == 3):
                    camelot_list.append(4)
                if (df['key'][i] == 4):
                    camelot_list.append(11)                
                if (df['key'][i] == 5):
                    camelot_list.append(6)                
                if (df['key'][i] == 6):
                    camelot_list.append(1)                
                if (df['key'][i] == 7):
                    camelot_list.append(8)                
                if (df['key'][i] == 8):
                    camelot_list.append(3)                
                if (df['key'][i] == 9):
                    camelot_list.append(10)                
                if (df['key'][i] == 10):
                    camelot_list.append(5)
                if (df['key'][i] == 11):
                    camelot_list.append(0)
            else:
                if (df['key'][i] == 0):
                    camelot_list.append(4)
                if (df['key'][i] == 1):
                    camelot_list.append(11)
                if (df['key'][i] == 2):
                    camelot_list.append(6)
                if (df['key'][i] == 3):
                    camelot_list.append(1)
                if (df['key'][i] == 4):
                    camelot_list.append(8)                
                if (df['key'][i] == 5):
                    camelot_list.append(3)                
                if (df['key'][i] == 6):
                    camelot_list.append(10)                
                if (df['key'][i] == 7):
                    camelot_list.append(5)                
                if (df['key'][i] == 8):
                    camelot_list.append(0)                
                if (df['key'][i] == 9):
                    camelot_list.append(7)                
                if (df['key'][i] == 10):
                    camelot_list.append(2)
                if (df['key'][i] == 11):
                    camelot_list.append(9)
        df['camelot'] = camelot_list
        return(df)

    ############################################
    ######        SPOTIFY  UTILITIES      ######
    ############################################
    
    def SearchSpotify(self, search_string):
        return self.spotify.search(search_string)
    

    
    def AuthorizeSpotify(self):
        print('Authorizing Spotify')
        self.user_name = "Ronald Marp"
        self.client_id = '1e05e910ef664f27949005267fdd0936'
        self.client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
        self.redirect_uri = "http://localhost/"
        self.scope = 'user-library-read'
        self.token = prompt_for_user_token(username=self.user_name,scope=self.scope,
                                           client_id=self.client_id,
                                           client_secret=self.client_secret,
                                           redirect_uri=self.redirect_uri)
        self.spotify = Spotify(auth=self.token)

In [6]:
## Percolator Meme - Teeth ['2QT5afoBT6nQabNuvcxXUa']

In [7]:
def search_for_artistID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]['artists'][0]
    print('Found artist %s' %result['name'])
    return(result['id'])

In [8]:
def search_for_trackID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]
    print('Found track %s by %s' %(result['name'], result['artists'][0]['name']))
    return(result['id'])

In [9]:
seed_artist_id = search_for_artistID('Aleksi Perala')

Found artist Aleksi Perälä


In [10]:
seed_track_id = search_for_trackID('Eckograph Hubot')

Found track Eckograph - Original Mix by Hubot


In [11]:
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id], 
                             #seed_artists=[seed_artist_id], 
                             seed_genres=['minimal-techno', 'techno']) #seed_genres=['minimal-techno', 'techno', 'detroit-techno'])

Authorizing Spotify


            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?response_type=code&client_id=1e05e910ef664f27949005267fdd0936&scope=user-library-read&redirect_uri=http%3A%2F%2Flocalhost%2F in your browser


Enter the URL you were redirected to: http://localhost/?code=AQCciVpTjsu0ra4xDSKDfZlGAa7T2Zd2bfCnRWESWI2zXIFDeT99MeFeGN5we6SnCiqq7lOxR9IA-nECw-gPvIhIC5FD2XXikrsBtZVCxc6LxkYVfFfIJnVrtTixsm_83hxzaFnylDRt2XAdWCsNyk3VrY2zbfN9-ZuNmZ8nZG9LQRMxsSGXQr_GJkPadf6smtSQ4avtbJ5Mcw




In [12]:
while len(d.set_list) < 10:
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool
Expanded track pool


,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
0,Triadz,Hubot,1,415633,35ug0mCea5X0LX7hxN0xQ2,0,Eckograph - Original Mix,3,0.019800,https://api.spotify.com/v1/audio-analysis/35ug...,0.718,415634,0.955,0.908,4,0.1090,-9.657,0,0.0541,125.030,4,https://api.spotify.com/v1/tracks/35ug0mCea5X0...,audio_features,spotify:track:35ug0mCea5X0LX7hxN0xQ2,0.0398,8
1,Driver,Alex Grekov,1,482982,6044lujukMaiEvRrOHJw7j,0,Driver - Xwired Remix,3,0.001230,https://api.spotify.com/v1/audio-analysis/6044...,0.741,482982,0.799,0.775,1,0.1690,-9.409,1,0.0764,128.007,4,https://api.spotify.com/v1/tracks/6044lujukMai...,audio_features,spotify:track:6044lujukMaiEvRrOHJw7j,0.3860,2
2,Richard Sen presents This Ain't Chicago - The ...,Baby Ford,1,328733,35VLWcRUFkCatwB6KsmYVd,0,Crashing,4,0.272000,https://api.spotify.com/v1/audio-analysis/35VL...,0.665,328733,0.676,0.942,6,0.0889,-10.400,1,0.0344,121.793,4,https://api.spotify.com/v1/tracks/35VLWcRUFkCa...,audio_features,spotify:track:35VLWcRUFkCatwB6KsmYVd,0.7320,1
3,Black Noise,Pantha Du Prince,1,397866,5EQJwmqtxNMTYN3Spr8NU3,43,Lay In A Shimmer,1,0.000815,https://api.spotify.com/v1/audio-analysis/5EQJ...,0.663,397867,0.639,0.836,0,0.5180,-12.280,1,0.0425,124.015,4,https://api.spotify.com/v1/tracks/5EQJwmqtxNMT...,audio_features,spotify:track:5EQJwmqtxNMTYN3Spr8NU3,0.0670,7
4,Energy Flow (DJ Koze Remixes),Mano Le Tough,1,511744,49qc7wqZapOcTKZcXS8FGt,42,Energy Flow - DJ Koze's Miles & More Remix,1,0.023600,https://api.spotify.com/v1/audio-analysis/49qc...,0.681,511745,0.415,0.929,10,0.1140,-14.002,0,0.0413,124.024,4,https://api.spotify.com/v1/tracks/49qc7wqZapOc...,audio_features,spotify:track:49qc7wqZapOcTKZcXS8FGt,0.2480,2
5,Straight Up Detroit Shit 2013,Underground Resistance,1,145853,6e0EKOW3cTsa7Wd100N43s,10,Timeline,3,0.000721,https://api.spotify.com/v1/audio-analysis/6e0E...,0.607,145853,0.947,0.937,10,0.0659,-5.233,0,0.1200,151.259,4,https://api.spotify.com/v1/tracks/6e0EKOW3cTsa...,audio_features,spotify:track:6e0EKOW3cTsa7Wd100N43s,0.9230,2
6,Orbit Head,Future Beat Alliance,1,426666,5tG08fZD8uACRk7pYySuGF,0,T3,2,0.000372,https://api.spotify.com/v1/audio-analysis/5tG0...,0.654,426667,0.794,0.768,7,0.1010,-10.873,1,0.0491,126.012,4,https://api.spotify.com/v1/tracks/5tG08fZD8uAC...,audio_features,spotify:track:5tG08fZD8uACRk7pYySuGF,0.1760,8
7,Minimal Impossible,Avrosse,1,335624,2Jy2QSICRcWTdsFamLFAAW,11,Minimal Impossible - Original Mix,1,0.002990,https://api.spotify.com/v1/audio-analysis/2Jy2...,0.804,335625,0.763,0.655,11,0.0211,-5.556,0,0.0504,128.004,4,https://api.spotify.com/v1/tracks/2Jy2QSICRcWT...,audio_features,spotify:track:2Jy2QSICRcWTdsFamLFAAW,0.0360,9
8,Kilimanjaro,Superpitcher,1,541613,6Es5126q11IzLOnhg2rFk7,23,Black Magic,9,0.000237,https://api.spotify.com/v1/audio-analysis/6Es5...,0.637,541613,0.524,0.737,2,0.1220,-12.687,0,0.0468,122.030,4,https://api.spotify.com/v1/tracks/6Es5126q11Iz...,audio_features,spotify:track:6Es5126q11IzLOnhg2rFk7,0.5370,6
9,Listen To Your Heart Part Three,Session Victim,1,302120,1PqtgPl8hcgQVSumslJ0G2,40,Almost Midnight,1,0.143000,https://api.spotify.com/v1/audio-analysis/1Pqt...,0.722,302120,0.425,0.719,8,0.2990,-15.142,1,0.1710,118.978,4,https://api.spotify.com/v1/tracks/1PqtgPl8hcgQ...,audio_features,spotify:track:1PqtgPl8hcgQVSumslJ0G2,0.6280,3


In [39]:
d.set_list['uri'].to_csv('../../../../spotify-downloader/output.txt', index=False)

In [ ]:
len(mixesdb)

In [ ]:
# Load and Process Datasets for big-data generation

spotify = pd.read_csv('data/spotify/spotify_database_mergedV2.csv')
del spotify['Unnamed: 0']
del spotify['Unnamed: 0.1']
spotify['artist_and_track'] = spotify['artist'] + ' ' + spotify['track']
print("Read and processed Spotify data")

discogs_m = pd.read_csv('data/discogs/discogs_mixed.csv')
discogs_r = pd.read_csv('data/discogs/discogs_releases.csv')
to_concat = [discogs_m, discogs_r]
discogs = pd.concat(to_concat)
del discogs_m
del discogs_r
del discogs['Unnamed: 0']
del discogs['genre']
del discogs['labels']
del discogs['length']
discogs['artist'] = discogs['artist'].apply(lambda x: re.sub(pattern='\([0-9]*\)', string=x, repl = '').strip())
discogs['artist_and_track'] = discogs['artist'] + ' ' + discogs['track']
print("Read and processed Discogs data")

mixesdb = pd.read_csv('data/mixesdb/mixesdb_all_sets_featurized.csv')
del mixesdb['Unnamed: 0']
mixesdb['artist_and_track'] = mixesdb['artist'] + ' ' + mixesdb['track']
print("Read and processed MixesDB data")


In [ ]:
spotify['popularity'].max()